In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### package loading

In [13]:
from typing import Awaitable
import numpy as np
from scipy import linalg as la
import time
import pickle

### module loading

In [14]:
def optimize(A, B):
    L = la.cholesky(A + np.eye(A.shape[1]) * 1e-5)
    y = la.solve_triangular(L.T, B, lower=True)
    u = la.solve_triangular(L, y, lower=False)
#     u = np.linalg.solve(A, B)
    return u


def iterationCPD(T, A1, A2, A3, A4, reg=1e-5):
    eye = np.eye(A1.shape[1])

    A1 = optimize(np.einsum('ir,im,jr,jm,kr,km->rm',A2,A2,A3,A3,A4,A4,optimize=True) + reg * eye, \
                  np.einsum('ijkl,jr,kr,lr->ri',T,A2,A3,A4,optimize=True)).T
    A2 = optimize(np.einsum('ir,im,jr,jm,kr,km->rm',A1,A1,A3,A3,A4,A4,optimize=True) + reg * eye, \
                  np.einsum('ijkl,ir,kr,lr->rj',T,A1,A3,A4,optimize=True)).T
    A3 = optimize(np.einsum('ir,im,jr,jm,kr,km->rm',A1,A1,A2,A2,A4,A4,optimize=True) + reg * eye, \
                  np.einsum('ijkl,ir,jr,lr->rk',T,A1,A2,A4,optimize=True)).T
    A4 = optimize(np.einsum('ir,im,jr,jm,kr,km->rm',A1,A1,A2,A2,A3,A3,optimize=True) + reg * eye, \
                  np.einsum('ijkl,ir,jr,kr->rl',T,A1,A2,A3,optimize=True)).T

    rec = np.einsum('ir,jr,kr,lr->ijkl',A1,A2,A3,A4)
    return A1, A2, A3, A4, rec


def Optimizer(Omega, A, RHS, num, reg=1e-5):
    """
    masked least square optimizer:
        A @ u.T = Omega * RHS
        number: which factor
        reg: 2-norm regulizer
    """
    N = len(A)
    R = A[0].shape[1]
    lst_mat = []
    T_inds = "".join([chr(ord('a')+i) for i in range(Omega.ndim)])
    einstr=""
    for i in range(N):
        if i != num:
            einstr+=chr(ord('a')+i) + 'r' + ','
            lst_mat.append(A[i])
            einstr+=chr(ord('a')+i) + 'z' + ','
            lst_mat.append(A[i])
    einstr+= T_inds + "->"+chr(ord('a')+num)+'rz'
    lst_mat.append(Omega)
    P = np.einsum(einstr,*lst_mat,optimize=True)
    o = np.zeros(RHS.shape)
    for j in range(A[num].shape[0]):
        o[j,:] = optimize(P[j], RHS[j,:])
    return o


def iterationPre(A1, A2, A3, A4, mask, T_, reg=1e-5):
    T_ = T_ * mask + (1-mask) * np.einsum('ir,jr,kr,lr->ijkl',A1,A2,A3,A4,optimize=True)
    eye = np.eye(A1.shape[1])
    A1 = optimize(np.einsum('ir,im,jr,jm,kr,km->rm',A2,A2,A3,A3,A4,A4,optimize=True) + reg * eye, \
                  np.einsum('ijkl,jr,kr,lr->ri',T_,A2,A3,A4,optimize=True)).T
    A2 = optimize(np.einsum('ir,im,jr,jm,kr,km->rm',A1,A1,A3,A3,A4,A4,optimize=True) + reg * eye, \
                  np.einsum('ijkl,ir,kr,lr->rj',T_,A1,A3,A4,optimize=True)).T
    A3 = optimize(np.einsum('ir,im,jr,jm,kr,km->rm',A1,A1,A2,A2,A4,A4,optimize=True) + reg * eye, \
                  np.einsum('ijkl,ir,jr,lr->rk',T_,A1,A2,A4,optimize=True)).T
    A4 = optimize(np.einsum('ir,im,jr,jm,kr,km->rm',A1,A1,A2,A2,A3,A3,optimize=True) + reg * eye, \
                  np.einsum('ijkl,ir,jr,kr->rl',T_,A1,A2,A3,optimize=True)).T
    return A1, A2, A3, A4


def iterationEMALS(mask, T, A, B, C, D, alpha=1, reg=1e-5, iters=1):
    """
    input:
        - mask: I x J x (K+1) x L
        - T:    I x J x (K+1) x L
        - |omega * (x - Adiag(c)B.TD)| + alpha * |mask * (X - [A, B, C_o, D])| + beta * reg
    """

    for i in range(iters):
        # get c
        c = Optimizer(mask[:,:,-1:,:], [A, B, np.random.random((1,R)), D], \
            np.einsum('ijkl,ir,jr,lr->kr',(T*mask)[:,:,-1:,:],A,B,D,optimize=True), 2, reg)

        eye = np.eye(R)
        # update A1, A2, A3, A4
        rec_X = mask * T + \
            (1-mask) * np.einsum('ir,jr,kr,lr->ijkl',A,B,np.concatenate([C, c], axis=0),D,optimize=True)
        A = optimize(alpha * np.einsum('ir,im,jr,jm,kr,km->rm',B,B,C,C,D,D,optimize=True) + \
                     np.einsum('ir,im,jr,jm,kr,km->rm',B,B,c,c,D,D,optimize=True) + reg * eye, \
                    alpha * np.einsum('ijkl,jr,kr,lr->ri',rec_X[:,:,:-1,:],B,C,D,optimize=True) + \
                     np.einsum('ijkl,jr,kr,lr->ri',rec_X[:,:,-1:,:],B,c,D,optimize=True)).T
        B = optimize(alpha * np.einsum('ir,im,jr,jm,kr,km->rm',A,A,C,C,D,D,optimize=True) + \
                     np.einsum('ir,im,jr,jm,kr,km->rm',A,A,c,c,D,D,optimize=True) + reg * eye, \
                    alpha * np.einsum('ijkl,ir,kr,lr->rj',rec_X[:,:,:-1,:],A,C,D,optimize=True) + \
                     np.einsum('ijkl,ir,kr,lr->rj',rec_X[:,:,-1:,:],A,c,D,optimize=True)).T
        C = optimize(np.einsum('ir,im,jr,jm,kr,km->rm',A,A,B,B,D,D) + reg * eye, \
                     np.einsum('ijkl,ir,jr,lr->rk',rec_X[:,:,:-1,:],A,B,D,optimize=True)).T
        D = optimize(alpha * np.einsum('ir,im,jr,jm,kr,km->rm',A,A,B,B,C,C,optimize=True) + \
                     np.einsum('ir,im,jr,jm,kr,km->rm',A,A,B,B,c,c,optimize=True) + reg * eye, \
                    alpha * np.einsum('ijkl,ir,jr,kr->rl',rec_X[:,:,:-1,:],A,B,C,optimize=True) + \
                     np.einsum('ijkl,ir,jr,kr->rl',rec_X[:,:,-1:,:],A,B,c,optimize=True)).T
    
    return A, B, np.concatenate([C, c], axis=0), D


def iterationCPCREF(mask, T, A, B, C, D, iters, reg=1e-5, tol=1e-5):
    T = T * mask
    I, J, K, L = T.shape
    _, R = A.shape
    A1 = np.random.random((I,R))
    A2 = np.random.random((J,R))
    A3 = np.random.random((K,R))
    A4 = np.random.random((L,R))
            
    rec = np.einsum('ir,jr,kr,lr->ijkl',A1,A2,A3,A4,optimize=True)
    for i in range(iters):
        A1 = Optimizer(mask, [A1, A2, A3, A4], np.einsum('ijkl,jr,kr,lr->ir',T,A2,A3,A4,optimize=True), 0, reg)
        A2 = Optimizer(mask, [A1, A2, A3, A4], np.einsum('ijkl,ir,kr,lr->jr',T,A1,A3,A4,optimize=True), 1, reg)
        A3 = Optimizer(mask, [A1, A2, A3, A4], np.einsum('ijkl,ir,jr,lr->kr',T,A1,A2,A4,optimize=True), 2, reg)
        A4 = Optimizer(mask, [A1, A2, A3, A4], np.einsum('ijkl,ir,jr,kr->lr',T,A1,A2,A3,optimize=True), 3, reg)
        rec2 = np.einsum('ir,jr,kr,lr->ijkl',A1,A2,A3,A4,optimize=True)
        if la.norm(rec - rec2) / la.norm(rec2) < tol:
            break
        else:
            rec = rec2
            
    return A1, A2, A3, A4


def iterationCPC(mask, T, A, B, C, D, reg=1e-5):
    T = T * mask
    I, J, K, L = T.shape
    _, R = A.shape
    
    A1, A2, A3, A4 = A, B, np.concatenate([C, np.random.random((1,R))], axis=0), D
    A1 = Optimizer(mask, [A1, A2, A3, A4], np.einsum('ijkl,jr,kr,lr->ir',T,A2,A3,A4,optimize=True), 0, reg)
    A2 = Optimizer(mask, [A1, A2, A3, A4], np.einsum('ijkl,ir,kr,lr->jr',T,A1,A3,A4,optimize=True), 1, reg)
    A3 = Optimizer(mask, [A1, A2, A3, A4], np.einsum('ijkl,ir,jr,lr->kr',T,A1,A2,A4,optimize=True), 2, reg)
    A4 = Optimizer(mask, [A1, A2, A3, A4], np.einsum('ijkl,ir,jr,kr->lr',T,A1,A2,A3,optimize=True), 3, reg)
    
    return A1, A2, A3, A4


def maskOptimizer(Omega, Omega_, A, A_, RHS, RHS_, num, alpha, reg=1e-5):
    """
    masked least square optimizer:
        A @ u.T = Omega * RHS
        number: which factor
        reg: 2-norm regulizer

    P is for normal left hand
    P2 is for historical left hand
    P3 is for historical right hand
    """
    
    N = len(A)
    R = A[0].shape[1]
    lst_mat, lst_mat_, lst_mat2, lst_mat3 = [], [], [], [A_[num]]
    T_inds = "".join([chr(ord('a')+i) for i in range(Omega.ndim)])
    einstr = ""
    for i in range(N):
        if i != num:
            if i == N-2:
                lst_mat.append(A_[-1]); lst_mat.append(A_[-1])
            else:
                lst_mat.append(A[i]); lst_mat.append(A[i])
            lst_mat_.append(A[i]); lst_mat_.append(A[i])
            einstr+=chr(ord('a')+i) + 'r' + ','
            lst_mat2.append(A[i]); lst_mat3.append(A_[i])
            einstr+=chr(ord('a')+i) + 'z' + ','
            lst_mat2.append(A[i]); lst_mat3.append(A[i])
    einstr2 = einstr[:-1] + "->rz"
    einstr3 = "tr," + einstr[:-1] + "->tz"
    einstr += T_inds + "->"+chr(ord('a')+num)+'rz'
    P2 = np.einsum(einstr2,*lst_mat2,optimize=True)
    lst_mat.append(Omega)
    P = np.einsum(einstr,*lst_mat,optimize=True)
    P3 = np.einsum(einstr3,*lst_mat3,optimize=True)
    o = np.zeros(RHS.shape)

    lst_mat_.append(Omega_)
    P_ = np.einsum(einstr,*lst_mat_,optimize=True)

    I = np.eye(R)
    for j in range(A[num].shape[0]):
        o[j,:] = np.linalg.inv(P[j] + P_[j] + alpha*P2 + reg*I) @ (RHS[j,:] + RHS_[j,:]  + alpha*P3[j,:])
    return o


def maskOptimizer2(Omega, A, A_, RHS, num, alpha, reg=1e-5):
    """
    masked least square optimizer:
        A @ u.T = Omega * RHS
        number: which factor
        reg: 2-norm regulizer

    P is for normal left hand
    P2 is for historical left hand
    P3 is for historical right hand
    """
    
    N = len(A)
    R = A[0].shape[1]
    lst_mat, lst_mat2, lst_mat3 = [], [], [A_[num]]
    T_inds = "".join([chr(ord('a')+i) for i in range(Omega.ndim)])
    einstr = ""
    for i in range(N):
        if i != num:
            lst_mat.append(A[i]); lst_mat.append(A[i])
            einstr+=chr(ord('a')+i) + 'r' + ','
            lst_mat2.append(A[i]); lst_mat3.append(A_[i])
            einstr+=chr(ord('a')+i) + 'z' + ','
            lst_mat2.append(A[i]); lst_mat3.append(A[i])
    einstr2 = einstr[:-1] + "->rz"
    einstr3 = "tr," + einstr[:-1] + "->tz"
    einstr += T_inds + "->"+chr(ord('a')+num)+'rz'
    P2 = np.einsum(einstr2,*lst_mat2,optimize=True)
    lst_mat.append(Omega)
    P = np.einsum(einstr,*lst_mat,optimize=True)
    P3 = np.einsum(einstr3,*lst_mat3,optimize=True)
    o = np.zeros(RHS.shape)

    I = np.eye(R)
    for j in range(A[num].shape[0]):
        o[j,:] = np.linalg.inv(P[j] + alpha*P2 + reg*I) @ (RHS[j,:] + alpha*P3[j,:])
    return o


def iterationGOCPT(mask, T, A, B, C, D, alpha=1, reg=1e-5):
    """
    input:
        - mask: I x J x (K+1)
        - T:    I x J x (K+1)
    |omega * (x - Adiag(c)B.T)| + alpha * |[A,B,C] - [A_o, B_o, C_o]| + beta * reg
    """

    A1, A2, A3, A4 = A.copy(), B.copy(), C.copy(), D.copy()
    mask_ = mask[:,:,-1:,:]
    T_ = T[:,:,-1:,:]
    # # get c
    c = Optimizer(mask_, [A1, A2, np.random.random((1,R)), A4], \
        np.einsum('ijkl,ir,jr,lr->kr',T_*mask_,A,B,D,optimize=True), 2, reg)

    _mask = mask[:,:,:-1,:]
    _T = T[:,:,:-1,:]
    eye = np.eye(R)
    A = maskOptimizer(mask_, _mask, [A, B, C, D], [A1, A2, A3, A4, c], \
                      np.einsum('ijkl,jr,kr,lr->ir',T_*mask_,B,c,D,optimize=True), \
                      np.einsum('ijkl,jr,kr,lr->ir',_T*_mask,B,C,D,optimize=True), 0, alpha, reg)
    B = maskOptimizer(mask_, _mask, [A, B, C, D], [A1, A2, A3, A4, c], \
                      np.einsum('ijkl,ir,kr,lr->jr',T_*mask_,A,c,D,optimize=True), \
                      np.einsum('ijkl,ir,kr,lr->jr',_T*_mask,A,C,D,optimize=True), 1, alpha, reg)
    C = maskOptimizer2(_mask, [A, B, C, D], [A1, A2, A3, A4], \
                       np.einsum('ijkl,ir,jr,lr->kr',_T*_mask,A,B,D,optimize=True), 2, alpha, reg)
    D = maskOptimizer(mask_, _mask, [A, B, C, D], [A1, A2, A3, A4, c], \
                      np.einsum('ijkl,ir,jr,kr->lr',T_*mask_,A,B,c,optimize=True), \
                      np.einsum('ijkl,ir,jr,kr->lr',_T*_mask,A,B,C,optimize=True), 3, alpha, reg)
    C = np.concatenate([C, c], axis=0)

    return A, B, C, D


def maskOptimizer3(Omega, A, A_, RHS, num, alpha, reg=1e-5):
    """
    masked least square optimizer:
        A @ u.T = Omega * RHS
        number: which factor
        reg: 2-norm regulizer

    P is for normal left hand
    P2 is for historical left hand
    P3 is for historical right hand
    """
    
    N = len(A)
    R = A[0].shape[1]
    lst_mat, lst_mat2, lst_mat3 = [], [], [A_[num]]
    T_inds = "".join([chr(ord('a')+i) for i in range(Omega.ndim)])
    einstr = ""
    for i in range(N):
        if i != num:
            if i == N-2:
                lst_mat.append(A_[-1]); lst_mat.append(A_[-1])
            else:
                lst_mat.append(A[i]); lst_mat.append(A[i])
            einstr+=chr(ord('a')+i) + 'r' + ','
            lst_mat2.append(A[i]); lst_mat3.append(A_[i])
            einstr+=chr(ord('a')+i) + 'z' + ','
            lst_mat2.append(A[i]); lst_mat3.append(A[i])
    einstr2 = einstr[:-1] + "->rz"
    einstr3 = "tr," + einstr[:-1] + "->tz"
    einstr += T_inds + "->"+chr(ord('a')+num)+'rz'
    P2 = np.einsum(einstr2,*lst_mat2,optimize=True)
    lst_mat.append(Omega)
    P = np.einsum(einstr,*lst_mat,optimize=True)
    P3 = np.einsum(einstr3,*lst_mat3,optimize=True)
    o = np.zeros(RHS.shape)

    I = np.eye(R)
    for j in range(A[num].shape[0]):
        o[j,:] = np.linalg.inv(P[j] + alpha*P2 + reg*I) @ (RHS[j,:] + alpha*P3[j,:])
    return o


def iterationGOCPTE(mask, T, A, B, C, D, alpha=1, reg=1e-5):
    """
    input:
        - mask: I x J x (K+1)
        - T:    I x J x (K+1)
    |omega * (x - Adiag(c)B.T)| + alpha * |[A,B,C] - [A_o, B_o, C_o]| + beta * reg
    """

    A1, A2, A3, A4 = A.copy(), B.copy(), C.copy(), D.copy()
    mask_ = mask[:,:,-1:,:]
    T_ = T[:,:,-1:,:]
    # # get c
    c = Optimizer(mask_, [A1, A2, np.random.random((1,R)), A4], \
        np.einsum('ijkl,ir,jr,lr->kr',T_*mask_,A,B,D,optimize=True), 2, reg)

    eye = np.eye(R)
    A = maskOptimizer3(mask_, [A, B, C, D], [A1, A2, A3, A4, c], \
                       np.einsum('ijkl,jr,kr,lr->ir',T_*mask_,B,c,D,optimize=True), 0, alpha, reg)
    B = maskOptimizer3(mask_, [A, B, C, D], [A1, A2, A3, A4, c], \
                       np.einsum('ijkl,ir,kr,lr->jr',T_*mask_,A,c,D,optimize=True), 1, alpha, reg)
    C = optimize(np.einsum('br,bm,cr,cm,dr,dm->rm',A,A,B,B,D,D,optimize=True) + reg * eye, \
                 np.einsum('ar,br,bm,cr,cm,dr,dm->ma',A3,A1,A,A2,B,A4,D,optimize=True)).T
    D = maskOptimizer3(mask_, [A, B, C, D], [A1, A2, A3, A4, c], \
                       np.einsum('ijkl,ir,jr,kr->lr',T_*mask_,A,B,c,optimize=True), 3, alpha, reg)
    C = np.concatenate([C, c], axis=0)

    return A, B, C, D


def metric(A, B, C, D, X, mask=None):
    rec = np.einsum('ir,jr,kr,lr->ijkl',A,B,C,D)
    if mask is not None:
        loss = la.norm(mask * (rec - X)) ** 2
        PoF = 1 - la.norm((1-mask) * (rec - X)) / la.norm((1-mask)*X) 
    else:
        loss = la.norm(rec - X) ** 2
        PoF = 1 - la.norm(rec - X) / la.norm(X) 
    return rec, loss, PoF

### configure

In [15]:
DATA = 'JHU'
perturb = 0 # add random perturbation or not

### run

In [19]:
# choose the dataset
if DATA == 'JHU':
    print (DATA)

    path = './exp-data/jhu_covid_tensor.pickle'
    data = pickle.load(open(path, 'rb'))
    X = np.concatenate([data[0], data[1]], axis=2)
    base, preIter, weight, sparsity, s = 0.5, 20, 2, 0.98, 0.05
    I, J, K, L = X.shape
    print (I, J, K, L)
    R = 5

elif DATA == 'claim':
    import pickle 
    path = './exp-data/patient_claim_2018_2019.pkl'
    X = pickle.load(open(path, 'rb'))
    base, preIter, weight, sparsity, s = 0.5, 20, 2, 0.98, 0.05
    I, J, K, L = X.shape
    R = 5
    
    
def set_random_seed(seed):
    np.random.seed(seed)
    # the initial tensor and the mask
    T = int(X.shape[2] * base) # should be larger than L
    mask_base = np.ones((I,J,T,L))
    for i in range(1,L):
        mask_base[:,:,-i,i:] = 0
    mask_list = [mask_base]
    for i in range(K-T):
        temp = np.concatenate([np.ones((I,J,1,L)), mask_list[-1]], axis=2)
        mask_list.append(temp)
    mask_list = mask_list[1:]


    """
    Preaparation with base mask
    """
    # initialization
    A = np.random.randn(I,R)
    B = np.random.randn(J,R)
    C = np.random.randn(T,R)
    D = np.random.randn(L,R)

    tic_pre = time.time()
    tic = time.time()
    result_pre, time_pre = [], []
    for i in range(preIter):
        A, B, C, D = iterationPre(A, B, C, D, mask_base, X[:,:,:T,:])
        toc = time.time()
        _, _, PoF = metric(A, B, C, D, X[:,:,:T,:], mask_base)
        result_pre.append(PoF); time_pre.append(time.time() - tic_pre)
        tic = time.time()
    print ('finish preparation')
    
    
    """
    EM-ALS
    """
    A_, B_, C_, D_, X_ = A.copy(), B.copy(), C.copy(), D.copy(), X.copy()
    T_ = T
    tic_EMALS = time.time()
    tic = time.time()
    result_EMALS, time_EMALS = [], []
    for index, mask_base_ in enumerate(mask_list):
        if perturb == 0:
            A_, B_, C_, D_ = iterationEMALS(mask_base_, X_[:,:,:T_+1,:], A_, B_, C_, D_, alpha=1, reg=1e-5)
        else:
            delta_mask = (np.random.random(X.shape) > sparsity)[:,:,:T_+1,:] * mask_base_
            delta_ratio = s - 2 * s * np.random.random(X.shape)
            X_[:,:,:T_+1,:] = X_[:,:,:T_+1,:] * (1 + delta_mask * delta_ratio[:,:,:T_+1,:])
            A_, B_, C_, D_ = iterationEMALS(mask_base_, X_[:,:,:T_+1,:], A_, B_, C_, D_, alpha=1, reg=1e-5)
        T_ += 1; toc = time.time()
        _, _, PoF = metric(A_, B_, C_, D_, X[:,:,:T_,:], mask_base_)
        result_EMALS.append(max(0, PoF)); time_EMALS.append(time.time() - tic_EMALS);
        tic = time.time()
    print ('finish EM-ALS')


    """
    CPC-ALS reference
    """
    A_, B_, C_, D_, X_ = A.copy(), B.copy(), C.copy(), D.copy(), X.copy()
    T_ = T
    tic_cpc_ref = time.time()
    tic = time.time()
    result_cpc_ref, time_cpc_ref = [], []
    for index, mask_base_ in enumerate(mask_list):
        if perturb == 0:
            A_, B_, C_, D_ = iterationCPCREF(mask_base_, X_[:,:,:T_+1,:], A_, B_, C_, D_, 50, reg=1e-5)
        else:
            delta_mask = (np.random.random(X.shape) > sparsity)[:,:,:T_+1,:] * mask_base_
            delta_ratio = s - 2 * s * np.random.random(X.shape)
            X_[:,:,:T_+1,:] = X_[:,:,:T_+1,:] * (1 + delta_mask * delta_ratio[:,:,:T_+1,:])
            A_, B_, C_, D_ = iterationCPCREF(mask_base_, X_[:,:,:T_+1,:], A_, B_, C_, D_, 50, reg=1e-5)
        T_ += 1; toc = time.time()
        _, _, PoF = metric(A_, B_, C_, D_, X[:,:,:T_,:], mask_base_)
        result_cpc_ref.append(max(0, PoF)); time_cpc_ref.append(time.time() - tic_cpc_ref)
        tic = time.time()
    print ('finish CPC-ALS reference')
    
    
    """
    CPC-ALS
    """
    A_, B_, C_, D_, X_ = A.copy(), B.copy(), C.copy(), D.copy(), X.copy()
    T_ = T
    tic_cpc = time.time()
    tic = time.time()
    result_cpc, time_cpc = [], []
    for index, mask_base_ in enumerate(mask_list):
        if perturb == 0:
            A_, B_, C_, D_ = iterationCPC(mask_base_, X_[:,:,:T_+1,:], A_, B_, C_, D_, reg=1e-5)
        else:
            delta_mask = (np.random.random(X.shape) > sparsity)[:,:,:T_+1,:] * mask_base_
            delta_ratio = s - 2 * s * np.random.random(X.shape)
            X_[:,:,:T_+1,:] = X_[:,:,:T_+1,:] * (1 + delta_mask * delta_ratio[:,:,:T_+1,:])
            A_, B_, C_, D_ = iterationCPC(mask_base_, X_[:,:,:T_+1,:], A_, B_, C_, D_, reg=1e-5)
        T_ += 1; toc = time.time()
        _, _, PoF = metric(A_, B_, C_, D_, X[:,:,:T_,:], mask_base_)
        result_cpc.append(max(0, PoF)); time_cpc.append(time.time() - tic_cpc)
        tic = time.time()
    print ('finish CPC-ALS')
    
    
    """
    GOCPT - E
    """
    A_, B_, C_, D_, X_ = A.copy(), B.copy(), C.copy(), D.copy(), X.copy()
    T_ = T

    tic_GOCPTE = time.time()
    tic = time.time()
    result_GOCPTE, time_GOCPTE, mse_GOCPTE, mape_GOCPTE = [], [], [], []
    for index, mask_base_ in enumerate(mask_list):
        if perturb == 0:
            A_, B_, C_, D_ = iterationGOCPTE(mask_base_, X_[:,:,:T_+1,:], A_, B_, C_, D_, weight / (base * K + index))
        else:
            delta_mask = (np.random.random(X.shape) > sparsity)[:,:,:T_+1,:] * mask_base_
            delta_ratio = s - 2 * s * np.random.random(X.shape)
            X_[:,:,:T_+1,:] = X_[:,:,:T_+1,:] * (1 + delta_mask * delta_ratio[:,:,:T_+1,:])
            A_, B_, C_, D_ = iterationGOCPTE(mask_base_, X_[:,:,:T_+1,:], A_, B_, C_, D_, weight / (base * K + index))
        T_ += 1; toc = time.time()
        _, _, PoF = metric(A_, B_, C_, D_, X[:,:,:T_,:], mask_base_)
        result_GOCPTE.append(max(0, PoF)); time_GOCPTE.append(time.time() - tic_GOCPTE)
        tic = time.time()
    print ('finish GOCPT E')
    
    
    """
    GOCPT
    """
    A_, B_, C_, D_, X_ = A.copy(), B.copy(), C.copy(), D.copy(), X.copy()
    T_ = T

    tic_GOCPT = time.time()
    tic = time.time()
    result_GOCPT, time_GOCPT, mse_GOCPT, mape_GOCPT = [], [], [], []
    for index, mask_base_ in enumerate(mask_list):
        if perturb == 0:
            A_, B_, C_, D_ = iterationGOCPT(mask_base_, X_[:,:,:T_+1,:], A_, B_, C_, D_, weight / 100 / (base * K + index))
        else:
            delta_mask = (np.random.random(X.shape) > sparsity)[:,:,:T_+1,:] * mask_base_
            delta_ratio = s - 2 * s * np.random.random(X.shape)
            X_[:,:,:T_+1,:] = X_[:,:,:T_+1,:] * (1 + delta_mask * delta_ratio[:,:,:T_+1,:])
            A_, B_, C_, D_ = iterationGOCPT(mask_base_, X_[:,:,:T_+1,:], A_, B_, C_, D_, weight / 100 / (base * K + index))
        T_ += 1; toc = time.time()
        _, _, PoF = metric(A_, B_, C_, D_, X[:,:,:T_,:], mask_base_)
        result_GOCPT.append(max(0, PoF)); time_GOCPT.append(time.time() - tic_GOCPT)
        tic = time.time()
    print ('finish GOCPT')

    return time_EMALS, result_EMALS, time_cpc_ref, result_cpc_ref, time_cpc, \
                result_cpc, time_GOCPTE, result_GOCPTE, time_GOCPT, result_GOCPT,

    
# report
for i in range(5):
    RESULT = set_random_seed(i)
    model = ['EM-ALS', 'CPC-ALS-REF', 'CPC-ALS', 'GOCPTE', 'GOCPT']
    print ('----- random seed {} -------'.format(i))
    for index in range(len(RESULT) // 2):
        print ('Model: {}, Time: {}, Avg. PoF: {}'.format(model[index], RESULT[2*index][-1], np.mean(RESULT[2*index+1])))
    print ()

    

In [35]:
# results without perturbation
EM-ALS & 1.68 $\pm$ 0.0013 & 0.68 $\pm$ 0.02453\\
CPC-ALS-REF & 18.21 $\pm$ 0.76 & 0.703 $\pm$ 0.01535\\
CPC-ALS & 2.14 $\pm$ 0.0013 & 0.6813 $\pm$ 0.02438\\
GOCPTE & 1.32 $\pm$ 0.0038 & 0.6897 $\pm$ 0.01642\\
GOCPT & 2.68 $\pm$ 0.002 & 0.692 $\pm$ 0.02429\\

# result with perturbation
EM-ALS & 2.13 $\pm$ 0.049 & 0.6622 $\pm$ 0.02427\\
CPC-ALS-REF & 18.45 $\pm$ 0.51 & 0.6905 $\pm$ 0.01085\\
CPC-ALS & 2.5 $\pm$ 0.013 & 0.6634 $\pm$ 0.02414\\
GOCPTE & 1.72 $\pm$ 0.034 & 0.6694 $\pm$ 0.005065\\
GOCPT & 3.17 $\pm$ 0.04 & 0.6827 $\pm$ 0.02413\\